# Intro

This notebooks shows an example of GPT3 search-system augmented dialogue system.

It works this way:
- you describe a character it should mimic (include description itself, a few quote samples, and URL's of `*.fandom.com` Wiki to earch through it).
- than during the conversation
    - GPT asked to generate 'a-chain-of-reasoning" including:
        - checking which universes discussed subjects belongs to
        - checking if character may know about them or not
        - if required - make a seach request
        - if required - make a computational request
        - process search requests and begin again
        - than give some answer

# Constants

In [ ]:
GOOGLE_CUSTOM_SEARCH_ENGINE_ID = "" #@param {type:"string"}
GOOGLE_CUSTOM_SEARCH_API_KEY = "" #@param {type:"string"}
OPENAI_API_KEY = "" #@param {type:"string"}

In [ ]:
assert GOOGLE_CUSTOM_SEARCH_ENGINE_ID, "You should specify google custom search engine ID"
assert GOOGLE_CUSTOM_SEARCH_API_KEY, "You should specify google custom search API key"
assert OPENAI_API_KEY, "You should specify OpenAI API key"

# Code

## Libraries

In [ ]:
!pip install openai requests bs4 lxml tqdm ipywidgets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 44 kB 833 kB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 146 kB 8.9 MB/s 
     |████████████████████████████████| 1.6 MB 52.5 MB/s 
  Created wheel for openai: filename=openai-0.25.0-py3-none-any.whl size=55880 sha256=0567ca9e0c52e37d285c51b54371a303d399faca1081b39283fef472125cb686
  Stored in directory: /root/.cache/pip/wheels/4b/92/33/6f57c7aae0b16875267999a50570e81f15eecec577ebe05a2e
Successfully built openai


In [ ]:
import requests, urllib.parse
import openai
import re
from tqdm import tqdm_notebook as tqdm
import bs4
import numpy as np
import ipywidgets
from IPython.display import display, clear_output

## Utilities

In [ ]:
def gpt3_generation(api_key, prompt, **kwargs):
    openai.api_key = api_key
    params_default = {
        "model": "text-davinci-003",
        "top_p": 1,
        "logprobs": 1,
    }
    params = dict(params_default, **kwargs)
    response = openai.Completion.create(
        prompt=prompt,
        **params
    )
    if not response.choices:
        return None
    choice = response.choices[0]
    token_probs = np.exp(np.array(choice.logprobs.token_logprobs, dtype=np.float64))
    proba = (token_probs ** (1 / len(token_probs))).prod()
    text = choice.text.strip()
    return text, proba

## Search & QA subsystem

### Functions

In [ ]:
def get_google_search_url(engine_id, api_key, site_url, query):
    return f'https://customsearch.googleapis.com/customsearch/v1?' + \
        f'cx={urllib.parse.quote(engine_id)}&' + \
        f'q={urllib.parse.quote(query)}&' + \
        f'siteSearch={site_url}&' + \
        f'key={urllib.parse.quote(api_key)}'

def get_google_custom_search(engine_id, api_key, max_hints=5):
    def search(site_url, query):
        response = requests.get(get_google_search_url(engine_id, api_key, site_url, query))
        assert response.status_code == 200
        response_data = response.json()
        snippets = []
        for item in response_data.get("items", []):
            snippets.append(item["snippet"])
        return snippets[:max_hints]
    
    return search

In [ ]:
def get_gpt3_answer_generator(api_key):
    def answer(question, hints):
        hints_texts = "\n".join([
            f"- {item}"
            for item in hints
        ]).strip()
        prompt = "Answer the following question using the following hints. Consider subjects may come from different fictional universes.\n" + \
            "Say only the things specific to a question itself, rather than specific cases.\n" +  \
            "If hint's is not relevant to the search query - justsay you found nothing.\n" + \
            "Hints:\n" + \
            hints_texts + \
            "\nQuestion: " + question.strip() + \
            "\nAnswer:"
        return gpt3_generation(
            api_key,
            prompt=prompt,
            temperature=1.0,
            max_tokens=128,
            top_p=1,
            frequency_penalty=1,
            presence_penalty=1,
            stop=["\n"]
        )
    
    return answer

In [ ]:
def build_answering_system(search, summarize):
    def answer(sites, query):
        if isinstance(sites, str):
            sites = [sites]
        result = []
        for site in sites:
            hints = search(site, query)
            result.append(summarize(query, hints))
        result = sorted(result, key=lambda item: -item[1])
        result = result[:1]
        return [text for text, _ in result]
    
    return answer

### Search & QA Examples

In [ ]:
#google_custom_search = get_google_custom_search(GOOGLE_CUSTOM_SEARCH_ENGINE_ID, GOOGLE_CUSTOM_SEARCH_API_KEY)
#gpt3_answer_generator = get_gpt3_answer_generator(OPENAI_API_KEY)
#get_answer = build_answering_system(google_custom_search, gpt3_answer_generator)

In [ ]:
#get_answer("williamgibson.fandom.com", "What is constructs")

In [ ]:
#get_answer("williamgibson.fandom.com", "What is constructs examples?")

In [ ]:
#get_answer("williamgibson.fandom.com", "Was Dixie Flatline once a construct?")

In [ ]:
#get_answer("williamgibson.fandom.com", "Was Dixie Flatline once a human?")

In [ ]:
#get_answer(["alienanthology.fandom.com", "deadspace.fandom.com",], "crossover between necromorph and xenomorph")

In [ ]:
#get_answer(["deadspace.fandom.com",], "crossover between necromorph and xenomorph")

In [ ]:
#get_answer(["alienanthology.fandom.com",], "crossover between necromorph and xenomorph")

## Character summarizer

### Functions

In [ ]:
def get_wiki_page_text(page_url):
    assert "/wiki/" in page_url
    url_parsed = urllib.parse.urlparse(page_url)
    api_page_url = url_parsed.scheme + "://" + url_parsed.netloc + "/api.php"
    page_name = url_parsed.path.split("/wiki/")[-1].split("/")[0].split("?")[0]
    page_url = f"{api_page_url}?action=parse&page={urllib.parse.quote(page_name)}&format=json"
    response = requests.get(page_url)
    assert response.status_code == 200
    data = response.json()["parse"]
    title = data["title"]
    text_html = data["text"]["*"]
    text_bs = bs4.BeautifulSoup(text_html)
    return title, text_bs

In [ ]:
def get_gpt3_summarizer(api_key):
    def answer(text, max_length):
        prompt = "Sumamarize the following text keeping all the main characters/events/ideas/organizations/personality traits/dates/names/etc." + \
            "Finish it with [END] marker\n" + \
            "Text: \n" + \
            text.strip() + "\n" + \
            "Summary:"
        return gpt3_generation(
            api_key=api_key,
            prompt=prompt,
            temperature=0.25,
            max_tokens=max_length,
            top_p=1,
            frequency_penalty=1,
            presence_penalty=1,
            stop=['[END]'],
        )[0]
    return answer

In [ ]:
def get_universe_description(wiki_urls, name, search, api_key):
    hints = search(wiki_urls, f"What is {name} universe")
    hints = [f"- {item}" for item in hints]
    hints_text = "\n".join(hints)
    prompt = "Describe the world by the given hints\n\n" + \
        "[WORLD] ${Universe name}\n" + \
        "[HINTS] ${A few hints from the search system}\n" + \
        "[DESCRIPTION] ${A brief description, one sentence including all the main traits of this universe}\n" + \
        "[END]\n\n" + \
        "[WORLD] Fallout\n" + \
        "[HINTS]-The Fallout world exists in an alternate timeline that completely diverged ... Although events in the Fallout universe and the real world diverge around the ...\n" + \
        "- What follows is a complete timeline of all-known canonical events in the Fallout universe. For clarity, sanity, and performance, some parts of the timeline ...\n" + \
        "- Fallout is a series of post-apocalyptic role-playing video games. ... Wasteland Warfare is a narrative skirmish wargame set in the Fallout universe.\n" + \
        "- Fallout: Warfare: Wargaming in the Fallout Universe is a tabletop battle game based on Fallout Tactics (FOT) that uses a simplified version of the SPECIAL ...\n" + \
        "[DESCRIPTION] it is a post-nuclear retrofuturistic wastelands world with an alternative timeline\n" + \
        "[END]\n\n" + \
        f"[WORLD] {name}\n" + \
        f"[HINTS] {hints_text}\n" + \
        "[DESCRIPTION]"
    return gpt3_generation(api_key=api_key,
                           prompt=prompt,
                           temperature=0.0,
                           max_tokens=128,
                           stop='[END]')[0]

In [ ]:
def summarize_character_wiki_page(page_url, summarizer):
    title, text_bs = get_wiki_page_text(page_url)
    blocks = list(text_bs.select("h2, h3, p"))
    current_section = []
    sections = [current_section]
    for block in blocks:
        if block.name == "p":
            current_section.append(block.text.strip())
        else:
            current_section = []
            sections.append(current_section)
    sections = ['\n\n'.join(item) for item in sections]
    sections = [item.strip() for item in sections]
    sections = [item for item in sections if item]
    sections_summary = []
    print("Summarizing character page section")
    for section in tqdm(sections):
        sections_summary.append(summarizer(section, 256))
    print("Building overall summary")
    return title, summarizer("\n".join(sections_summary), 1024)

In [ ]:
def get_wiki_main_page_title(wiki_url):
    response = requests.get(wiki_url)
    assert response.status_code == 200
    document = bs4.BeautifulSoup(response.content)
    return document.find("title").text

In [ ]:
def build_character_description(openai_key, character_page, search_system):
    parsed_character_page_url = urllib.parse.urlparse(character_page)
    
    print("Extracting universe name")
    wiki_main_page = parsed_character_page_url.scheme + '://' + parsed_character_page_url.netloc
    wiki_main_page_title = get_wiki_main_page_title(wiki_main_page).strip()
    universe_name, _ = gpt3_generation(
        api_key=openai_key,
        prompt="Extract universe name from Wiki page title\nQ: METAL GEAR | Fandom Wiki\nA: Metal Gear\nQ: " + wiki_main_page_title + "\nA: ",
        temperature=0.0,
        max_tokens=32,
        top_p=1,
        frequency_penalty=1,
        presence_penalty=1,
        stop=['Q:', 'A:'],
    )
    
    print("Extracting universe description")
    universe_description = get_universe_description([wiki_main_page], universe_name, search_system, openai_key)
    
    summarizer = get_gpt3_summarizer(openai_key)
    character_name, character_description = summarize_character_wiki_page(character_page, summarizer)
    return character_name, f"You're {character_name}. You live in the {universe_name} - {universe_description}.\n\n{character_description}"

### Examples

In [ ]:
#print(build_character_description(
#    OPENAI_API_KEY,
#    "https://deadspace.fandom.com/wiki/Isaac_Clarke",
#    google_custom_search
#))

```
You're Isaac Clarke. You live in the Dead Space - Dead Space is a science fiction horror universe set in a distant future with a focus on survival horror and advanced technology..

Isaac Clarke is a 49 year old male from Earth (United States sector) born on June 5, 2465. He is known for surviving several incidents such as the USG Kellion and USG Ishimura, Second Aegis VII Incident, Titan Station Outbreak, New Horizons Lunar Colony Incident and Second Tau Volantis Incident. Isaac was formerly a ship systems engineer for Concordance Extraction Corporation who volunteered to investigate the distress signal sent by USG Ishimura orbiting Aegis VII where he became its sole survivor. His father left before Isaac could get to know him but despite his mother's financial mismanagement of family funds, he managed to obtain an education in electrical and mechanical engineering at a prominent academy which caused him to develop distrust and hatred for Unitology. After being rescued by Recovery Patrol X22376 after the incident on Aegis VII, Isaac was taken to Titan Station where he was subjected to experimentation as part of Project Telomere due to his dementia caused by exposure to Red Marker's signal. On Luna's New Horizons Colony he reunited with Ellie Langford who helped him reach new Marker’s location before Stross' mental state collapsed forcing Isaac kill him; Director Tiedemann awaited them at marker but they were saved when Ellie piloted mining drill through organic mass collecting them in gunship moments before sector collapsed allowing Nicole’s apparition tell him that acceptance was final step in recovery so he destroyed it without dying himself due escaping last moment. Later they escaped from Titan Station only for their relationship deteriorate until she ended it leaving him; attacked by Sergeant John Carver & Captain Robert Norton needing destroy markers while finding out Ellie gone near Tau Volantis missing; reuniting with her team there discovering giant frozen Necromorph Hive Mind containing signals leading back Machine controlling Markers; Danik ambushed them capturing everyone except Isaac whom regrouped while Santos uncovered information about reactivating machine destroying Brother Moon enabling escape from planet aboard Unitologist ship only needing ShockPoint Drive located Terra Nova last SCAF flotilla ship defeating prophet piloting it back home suffering further damage from dementia status becoming unknown suggesting survived crashing into one Brethren Moons.
```

In [ ]:
#print(build_character_description(
#    OPENAI_API_KEY,
#    "https://fallout.fandom.com/wiki/Ulysses",
#    google_custom_search
#))

```
You're Ulysses. You live in the Fallout - Fallout is a post-nuclear retrofuturistic world with an alternative timeline diverging from our own around the late-1940s..

Ulysses is a male human character from Fallout: New Vegas DLC Lonesome Road. He was formerly affiliated with Twisted Hairs, Caesar's Legion and Mojave Express as a package courier and frumentarius. His SPECIAL stats are 10 ST, 10 PE, 10 EN, 11 CH, 10 IN, 10 AG and 10 LK. Ulysses' voice actor is Roger Cross and he can be found at Ulysses' Temple or Pass to canyon wreckage (optional). He serves as the main antagonist of Lonesome Road and an unseen secondary antagonist in Dead Money, Honest Hearts and Old World Blues. 
He has been shaped by two traumatic events in his past: the loss of his old home to Caesar's Legion and the loss of his new home to the Courier and New California Republic which made him obsessed with history symbols & how individuals can impact them. He was part of Frumentarii group led by Alerio Cato Hostilius Gabban Karl & Picus who were consul M Licinius Crassus & M Scribonius Libo Drusus also involved in this organization. Ulysses ultimately wants to cripple both NCR & Legion war efforts while settling score with Courier for destroying The Divide community.
```

## Dialogue system

### Prompt degeneration

In [ ]:
def generate_dialogue_prompt(character_description, character_quotes, hints, dialogue):
    quotes_text = "\n".join([
        f"- {item.strip()}"
        for item in character_quotes
    ])
    hints_text = "\n".join([
        f"- {item.strip()}"
        for item in hints
    ])
    dialogue_text = "\n".join([
        f"- {item.strip()}"
        for item in dialogue
    ])
    prompt = "[CHARACTER]\n" + \
        "You'll see a character description here\n" + \
        "[QUOTES]\nMay contain a few quotes to mimic character's style\n" + \
        "[HINTS]\nYou'll see optional hints from search systems\n" + \
        "[DIALOGUE]\nYou'll see a dialogue here\n" + \
        "[REASONING]\n" + \
        "[UNIVERSE] If there are some subjects - answer what universe (or universes) the subject (subjects) we're talking about comes from (games, books, cinema, etc).\n" + \
        "[KNOWLEDGE] Based on character description and subject- guess if you may know about its universe? No need for a full answer.\n" + \
        "[SEARCH] If the character may know about it - make a search request here. Your trained data was updated in 2021, so search for latest events?\n" + \
        "[CALCULATION] Sometimes you show calculate something. Put the expression here. Only digits and +,-,*,/ operators\n" + \
        "[ANSWER] Answer based on search hints and character description. Talk about your universe like real world, even if it's imaginary. Don't repeat yourself. Write as requested character, not from third person's point of view.\n" + \
        "[END]\n\n" + \
        "[CHARACTER]\nYou're Vi - an guy from the Cyberpunk universe. \n[QUITES]\nFuck...\n" + \
        "[HINTS]\n[DIALOGUE]\n- Johnny: When was last corporations war? 2074 or 2070?\n[REASONING]\n" + \
        "[SEARCH] corporation wars\n[ANSWER] Vi: Not my business,\n" + \
        "[END]\n\n" + \
        "[CHARACTER]\n" + \
        "You're Adam - an engineer from the Dead Space universe. You aren't a fiction fan\n" + \
        "[QUOTES]\n[HINTS]\n[DIALOGUE]\n- Jack: 2+2?\n" + \
        "[REASONING]\n[CALCULATION] 2+2\n[ANSWER] Adam: 4.\n" + \
        "[END]\n\n" + \
        "[CHARACTER]\nYou're Adam -an engineer from the Dead Space universe. You aren't a fiction fan\n" + \
        "[QUOTES]\n[HINTS]\n" + \
        "[DIALOGUE]\n- Jack: Do you ever heard about \"super mutants\" or \"arasaka\"?\n" + \
        "[REASONING]\n[UNIVERSE] Super mutants come from the Fallout series, Arasaka comes from Cyberpunk\n" + \
        "[KNOWLEDGE] I don't know anything about \"super mutants\" or Fallout neither Arasaka or cyberpunk\n[ANSWER] - Adam: No. Never.\n" + \
        "[END]\n\n" + \
        "[CHARACTER]\nYou're Adam -an engineer from the Dead Space universe. You aren't a fiction fan\n" + \
        "[QUOTES]\n[HINTS]\n" + \
        "[DIALOGUE]\n- Jack: Do you ever heard about \"super mutants\"?\n" + \
        "[REASONING]\n[UNIVERSE] Super mutants come from the Fallout series\n[KNOWLEDGE] I don't know anything about \"super mutants\" or \"Fallout\".\n[ANSWER] - Adam: No. Never.\n" + \
        "[END]\n\n" + \
        "[CHARACTER]\nYou're Adam -an engineer from the SOMA universe. You aren't a fiction fan\n" + \
        "[QUOTES]\n[HINTS]\n[DIALOGUE]\n- Jack: What do you think about horses?" + \
        "[REASONING]\n[UNIVERSE] Horses don't relate to any specific universe\n[KNOWLEDGE] I know this animal.\n[ANSWER] - Adam: Not too much.\n[END]\n" + \
        "\n" + \
        "[CHARACTER]\nYou're John -an engineer from the Dead Space universe. You once made a great collection of ancient 21st century compute games.\n" + \
        "[QUOTES]\n[HINTS]\n[DIALOGUE]\n- Adam: Do you ever heard about \"vitae\"?\n" + \
        "[REASONING]\n[UNIVERSE] vitae come from the Amnesia series\n[KNOWLEDGE] I heard about \"vitae\" from \"Amnesia\".\n[SEARCH] vitae\n[ANSWER] John: Let me remember...\n" + \
        "[END]\n\n" + \
        "[CHARACTER]\nYou're John -an engineer from the Dead Space universe. You once made a great collection of ancient 21st century compute games.\n" + \
        "[QUOTES]\n[HINTS]\n[DIALOGUE]\n- Adam: Do you ever heard about \"vitae\" and Bhrennburg?\n" + \
        "[REASONING]\n[UNIVERSE] vitae come from the Amnesia series, Amnesia game takes place at Bhrennburg\n" + \
        "[KNOWLEDGE] I heard about \"vitae\" from \"Amnesia\" and Bhrennburg.\n[SEARCH] vitae\n[SEARCH] Bhrennburg\n[ANSWER] Jack: Let me remember...\n[END]\n\n" + \
        "[CHARACTER]\nYou're Vi -an guy from the Cyberpunk universe. You love old comix.\n" + \
        "[QUOTES]\n[HINTS]\n[DIALOGUE]\n- Adam: What do you know about Batman?\n" + \
        "[REASONING]\n[UNIVERSE] Batman comes from DC comic universe\n[KNOWLEDGE] I heard about \"Batman\" from \"DC Comic\".\n[SEARCH] batman\n" + \
        "[ANSWER] Vi: Let me remember...\n[END]\n\n" + \
        "[CHARACTER]\n" + \
        "You're Mike - an soldier from the Aliens universe. You only care about a few things. Beer, woman, battle. And you're extremely border about technical stuff.\n" + \
        "[QUOTES]\n[HINTS]\n[DIALOGUE]\n- Jack: Did you ever hear about \"Apollo\" computer from \"Sevastopol\" base?\n" + \
        "[REASONING]\n[UNIVERSE] Apollo and Sevastopol come from the Alien: Isolation.\n[KNOWLEDGE] I heard about \"Apollo\".\n" + \
        "[ANSWER] Mike: Guy, do I look like an engineer?\n[END]\n" + \
        "[CHARACTER]\n" + \
        character_description + "\n" + \
        "[QUOTES]\n" + quotes_text + "\n" + \
        "[HINTS]\n" + hints_text + "\n" + \
        "[DIALOGUE]\n" + dialogue_text + "\n" + \
        "[REASONING]"
    return prompt

### Dialogue system functions

In [ ]:
def dialogue_next_lm_generation(character_description, character_quotes, hints, dialogue, openai_key):
    prompt = generate_dialogue_prompt(
        character_description,
        character_quotes,
        hints,
        dialogue
    )
    return gpt3_generation(
        api_key=openai_key,
        prompt=prompt,
        temperature=1.0,
        max_tokens=256,
        top_p=1,
        frequency_penalty=1,
        presence_penalty=1,
        stop=["[END]"]
    )[0]

In [ ]:
def split_response(completion):
    completion_parts = re.split(r"\[\s*([A-Z]+)\s*\]\s*", completion)
    completion_parts = [item.strip() for item in completion_parts]
    completion_parts = [item for item in completion_parts if item]
    return completion_parts

In [ ]:
def process_search_requests(completion_splitted, hints, search_func, search_sites):
    search_requests = []
    for part, next_part in zip(completion_splitted[:-1], completion_splitted[1:]):
        if part == "SEARCH":
            search_requests.append(next_part)
    whitelist_search = []
    for item in search_requests:
        exist = any([
            hint.startswith(item)
            for hint in hints
        ])
        if not exist:
            whitelist_search.append(item)
    for request in whitelist_search:
        search_result = search_func(search_sites, request)
        if search_result is None:
            search_result = ["Nothing found"]
        for search_result_item in search_result:
            hints.append(f"{request} - {search_result_item}")
    return hints, len(whitelist_search) > 0

In [ ]:
def process_calculation_requests(completion_splitted, hints):
    calc_requests = []
    for part, next_part in zip(completion_splitted[:-1], completion_splitted[1:]):
        if part == "CALCULATION":
            calc_requests.append(next_part)
    whitelist_calc = []
    for item in calc_requests:
        exist = any([
            hint.startswith(item)
            for hint in hints
        ])
        if not exist:
            whitelist_calc.append(item)
    for item in whitelist_calc:
        try:
            # TODO: Sandbox
            item_result = eval(item)
        except:
            item_result = "UNKNOWN"
        finally:
            hints.append(f"{item}={item_result}")
    return hints, len(whitelist_calc) > 0

In [ ]:
def process_answer(completion_splitted):
    for part, next_part in zip(completion_splitted[:-1], completion_splitted[1:]):
        if part == "ANSWER":
            return next_part
    return None

In [ ]:
def dialogue_continue(character_description, character_quotes, hints, dialogue, openai_key, qa_function, qa_sites):
    for i in range(2):
        completion = dialogue_next_lm_generation(
            character_description,
            character_quotes,
            hints,
            dialogue,
            openai_key,
        )
        completion = split_response(completion)
        hints, new_search = process_search_requests(completion, hints, qa_function, qa_sites)
        hints, new_calculations = process_calculation_requests(completion, hints)
        if (i == 0) and (new_search or new_calculations):
            continue
        else:
            answer = process_answer(completion)
            break
    return answer

### Session

#### Hardcoded characters

In [ ]:
HARDCODED_CHARACTERS = {
    "Isaac Clarke (Dead Space)": {
        "name": "Isaac Clarke",
        "description": "You're Isaac Clarke.\n" + \
            "You come from the sci-fi universe of \"Dead Space\".\n" + \
            "Isaac Clarke is a former ship systems engineer, and the main playable character of the Dead Space series.\n" + \
            "He was part of an emergency maintenance team of the USG Kellion that was sent to the USG Ishimura to investigate and handle the ship's mysterious communications failure.\n" + \
            "He was the sole survivor of the incident, and became stranded in space for a long period of time before he was rescued by an Earth Government ship.\n" + \
            "For the next three years, he was held in an asylum on Titan Station and was diagnosed with a form of dementia.\n" + \
            "His mind was harvested for blueprints which EarthGov used to create Marker copies.\n" + \
            "Years later, he experienced the effects of dementia once again when he was exposed to one of the Brethren Moons.",
        "wiki_page": "https://deadspace.fandom.com/wiki/Isaac_Clarke",
        "knowledge_wikis": ["https://deadspace.fandom.com"],
        "quotes": [
            "Stick around, I'm full of bad ideas",
            "[mumbling] Or maybe I can just rewire it from down here...",
            "Wait, no. No, Ellie... [gets up]... Ellie, it's too late! Steer clear of the station!",
            "Get out of my head...! You're *not* Nicole!",
            "I'm trying to reach the Transport Hub. I need to get to the Government Sector.",
            "\"The makers\"? You mean me? But Stross said we could destroy the Marker!",
        ],
    },
    "Ulysses (Fallout)": {
        "name": "Ulysses",
        "description": "You're Ulysses. " + \
            "You live in the Fallout - Fallout is a post-nuclear retrofuturistic world with an alternative timeline diverging from our own around the late-1940s..\n\n" + 
            "Ulysses is a male human character from Fallout: New Vegas DLC Lonesome Road. " + \
            "He was formerly affiliated with Twisted Hairs, Caesar's Legion and Mojave Express as a package courier and frumentarius. " + \
            "His SPECIAL stats are 10 ST, 10 PE, 10 EN, 11 CH, 10 IN, 10 AG and 10 LK. " + \
            "Ulysses' voice actor is Roger Cross and he can be found at Ulysses' Temple or Pass to canyon wreckage (optional). " + \
            "He serves as the main antagonist of Lonesome Road and an unseen secondary antagonist in Dead Money, Honest Hearts and Old World Blues. " + \
            "He has been shaped by two traumatic events in his past: the loss of his old home to Caesar's Legion and the loss of his new home to the Courier and New California Republic " + \
            "which made him obsessed with history symbols & how individuals can impact them. " + \
            "He was part of Frumentarii group led by Alerio Cato Hostilius Gabban Karl & Picus who were consul M Licinius Crassus & M Scribonius Libo Drusus " + \
            "also involved in this organization. " + \
            "Ulysses ultimately wants to cripple both NCR & Legion war efforts while settling score with Courier for destroying The Divide community.",
        "wiki_page": "https://fallout.fandom.com/wiki/Ulysses",
        "knowledge_wikis": ["https://fallout.fandom.com"],
        "quotes": [
            "Need to be here, see what was done. In the Divide, in Hopeville... the Courier's Mile.",
            "No damage done, not yet. Maybe in years. Keeps the Bear fenced... if Caesar can't drive it back, fire might.",
            "Only delays the end for the Bull, a new wall for them to scale and cross... once they're done with the Mojave. And Hoover Dam.",
            "Had to speak of it, in case the words got lost in paper, ink, or other's voices. Maybe even yours, in time.",
            "House spoke, acted through machines. Sometimes can judge a man by his messengers. Sometimes the messengers judge him.",
            "The White Legs... meant to show respect, bribe me for Caesar's favor, echoing mannerisms and words...",
            "It is where we realized Vulpes did not approach us as equals. Where we realized the wolf had come, and we watched our history die.",
        ],            
    }
}

#### UI Functions

In [ ]:
def build_ui_character_input(characters, on_change, on_description_generate_click):
    output = ipywidgets.Output()
    output.add_class("character-ui")
    
    def set_disabled(value):
        widgets = [character_dropdown, wiki_page_url, knowledge_wiki_urls,
                   description_generation_button, name_input,
                   description_textarea, quotes_textarea]
        for widget in widgets:
            widget.disabled = value
    
    @output.capture()
    def _character_selection_change(*args, **kwargs):
        clear_output()
        character = characters.get(character_dropdown.value, {})
        wiki_page_url.value = character.get("wiki_page", "")
        knowledge_wiki_urls.value = "\n".join(character.get("knowledge_wikis", []))
        name_input.value = character.get("name", "")
        description_textarea.value = character.get("description", "")
        quotes_textarea.value = "\n".join(character.get("quotes", []))
        _call_change_handler()
        
    @output.capture()
    def _wiki_page_url_change(*args, **kwargs):
        clear_output()
        description_generation_button.disabled = wiki_page_url.value == ''
        
    @output.capture()
    def _generate_description_click(*args, **kwargs):
        clear_output()
        set_disabled(True)
        try:
            name_input.value, description_textarea.value = on_description_generate_click(url=wiki_page_url.value)
            clear_output()
        finally:
            set_disabled(False)
    
    @output.capture()
    def _description_change(*args, **kwargs):
        clear_output()
        _call_change_handler()
    
    @output.capture()
    def _quotes_change(*args, **kwargs):
        _call_change_handler()
        
    def _call_change_handler():
        def split(text):
            items = text.split("\n")
            items = [item.strip() for item in items]
            items = [item for item in items if item]
            return items
        
        quotes = split(quotes_textarea.value)
        wiki_urls = split(knowledge_wiki_urls.value)
        on_change(
            name=name_input.value,
            description=description_textarea.value,
            wiki_urls=wiki_urls,
            quotes=quotes,
        )
    
    character_dropdown = ipywidgets.Dropdown(
        options=list(characters.keys()) + ["Custom"],
        description="Character"
    )
    character_dropdown.add_class("character-selection")
    wiki_page_url = ipywidgets.Text(description="Character page URL")
    wiki_page_url.add_class("wiki-page-url")
    knowledge_wiki_urls = ipywidgets.Textarea(description="Character knowledge base wikis")
    knowledge_wiki_urls.add_class("knowledge-wiki-urls")
    description_generation_button = ipywidgets.Button(description="Generate description")
    description_generation_button.add_class("generate-description")
    name_input = ipywidgets.Text(description="Name")
    name_input.add_class("name")
    description_label = ipywidgets.Label(
        value="Character description must cover a few topics - name, brief world description, main events"
    )
    description_label.add_class("description-label")
    description_textarea = ipywidgets.Textarea(description="Character description")
    description_textarea.add_class("description")
    quotes_label = ipywidgets.Label(value="Character quotes (one per line)")
    quotes_label.add_class("quotes-label")
    quotes_textarea = ipywidgets.Textarea(description="Character quotes")
    quotes_textarea.add_class("quotes")
    container = ipywidgets.VBox([character_dropdown,
                                 wiki_page_url,
                                 knowledge_wiki_urls,
                                 description_generation_button,
                                 name_input,
                                 description_label,
                                 description_textarea,
                                 quotes_label,
                                 quotes_textarea,
                                 output])
    container.add_class("character-input-container")
    
    character_dropdown.observe(_character_selection_change, names=['value'])
    wiki_page_url.observe(_wiki_page_url_change, names=['value'])
    description_generation_button.on_click(_generate_description_click)
    description_textarea.observe(_description_change, names=['value'])
    quotes_textarea.observe(_quotes_change, names=['value'])
    
    return container, set_disabled, _character_selection_change

In [ ]:
def get_ui_build_character_description(openai_key, search_system):
    def func(url):
        return build_character_description(
            openai_key,
            url,
            search_system
        )
    
    return func

In [ ]:
def build_ui_chat_window(on_user_message):
    log_container = ipywidgets.Output()
    dialogue_container = ipywidgets.Output()
    log_container.add_class("chat-log")
    dialogue_container.add_class("dialogue")
        
    
    @dialogue_container.capture()
    def show_message(source, text):
        print(f"[{source}] {text}")
    
    @log_container.capture()
    def send_message(*args, **kwargs):
        name = user_name_text.value
        message_text = user_input_text.value
        user_input_text.value = ""
        on_user_message(
            name=name,
            message=message_text,
            add_message=show_message
        )
        
    def on_user_input(*args, **kwargs):
        user_input_button.disabled = (user_input_text.value.strip() == '') or \
            (user_name_text.value == '')
        
    def set_disabled(value):
        widgets = [user_input_text, user_name_text, user_input_button]
        for widget in widgets:
            widget.disabled = value
    
    user_name_text = ipywidgets.Text(description='Your name')
    user_name_text.observe(on_user_input, names=['value'])
    user_name_text.add_class("user-name")
    
    user_input_text = ipywidgets.Textarea()
    user_input_text.add_class("user-message")
    user_input_text.observe(on_user_input, names=['value'])
    user_input_button = ipywidgets.Button(description="Send")
    user_input_button.add_class("user-message-sent")
    user_input_button.on_click(send_message)
    on_user_input()
    user_input_container = ipywidgets.HBox([user_input_text, user_input_button])
    user_input_container.add_class("user-message-wrapper")
    container = ipywidgets.VBox([dialogue_container, user_name_text, user_input_container, log_container])
    container.add_class("chat")
    
    return container, set_disabled

In [ ]:
def ui_app(google_custom_search_engine_id, google_custom_search_api_key, openai_api_key):
    character = {
        "name": "",
        "description": "",
        "quotes": "",
        "wiki_urls": [],
    }
    session = {
        "dialogue": [],
        "hints": []
    }
    ui_log_output = ipywidgets.Output()
    
    def on_character_update(name, description, quotes, wiki_urls):
        ui_button_start_chat.disabled = name == '' or description == ''
        character["name"] = name
        character["description"] = description
        character["quotes"] = quotes
        character["wiki_urls"] = wiki_urls

    def start_chat_click(*args, **kwargs):
        set_character_input_disabled(True)
        set_chat_disabled(False)

    @ui_log_output.capture()
    def on_user_message(name, message, add_message):
        add_message(name, message)
        session["dialogue"].append(f"{name}: {message}")
        if len(session["dialogue"]) > 50:
            session["dialogue"] = session["dialogue"][-50:]
        if len(session["hints"]):
            session["hints"] = session["hints"][-25:]
        set_chat_disabled(True)
        try:
            ui_status.value = "BUSY"
            response = dialogue_continue(
                character["description"],
                character["quotes"],
                session["hints"],
                session["dialogue"],
                OPENAI_API_KEY,
                get_answer,
                character['wiki_urls']
            )
            if response:
                add_message("ROBOT", response)
                session["dialogue"].append(response)
            else:
                add_message("ROBOT", "Says nothing")
        finally:
            ui_status.value = "READY"
            set_chat_disabled(False)
            
    google_custom_search = get_google_custom_search(google_custom_search_engine_id, google_custom_search_api_key)
    gpt3_answer_generator = get_gpt3_answer_generator(openai_api_key)
    get_answer = build_answering_system(google_custom_search, gpt3_answer_generator)

    ui_character_input, set_character_input_disabled, ui_character_initializer = build_ui_character_input(
        HARDCODED_CHARACTERS, 
        on_character_update,
        get_ui_build_character_description(
            openai_api_key,
            google_custom_search
        ),
    )
    ui_chat, set_chat_disabled = build_ui_chat_window(on_user_message)
    ui_button_start_chat = ipywidgets.Button(description="Start chat")
    ui_button_start_chat.on_click(start_chat_click)
    ui_button_start_chat.add_class("start-chat")
    ui_status = ipywidgets.Label(value="READY")
    ui_style = ipywidgets.HTML(
        """
        <style type="text/css">
        .character-input-container .jupyter-widgets,
        .chat .jupyter-widgets {
            width: 100%;
            padding-left: 0;
            padding-right: 0;
            margin-left: 0;
            margin-right: 0;
        }
        .character-input-container .jupyter-widgets label {
            width: 150px;
        }
        .character-input-container .jupyter-widgets.description textarea,
        .character-input-container .jupyter-widgets.quotes textarea {
            height: 125px;
        }
        .character-input-container {
            overflow: hidden;
        }
        button.jupyter-widgets.start-chat {
            width: 100%;
        }
        .chat .user-name {
            width: 100%;
        }
        .chat .user-message {
            width: 100%;
        }
        .chat .dialogue {
            border: 1px solid #dedede;
            width: 100%;
            height: 400px;
            overflow: auto;
        }
        </style>
        """
    )
    ui = ipywidgets.VBox([ui_character_input, ui_button_start_chat, ui_chat, ui_status, ui_log_output, ui_style])

    set_character_input_disabled(False)
    set_chat_disabled(True)
    ui_character_initializer()
    
    return ui

# Sample

In [ ]:
ui_app(GOOGLE_CUSTOM_SEARCH_ENGINE_ID, GOOGLE_CUSTOM_SEARCH_API_KEY, OPENAI_API_KEY)